<a href="https://colab.research.google.com/github/DmitryKutsev/DeepHW/blob/master/my_hw3_attn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install youtokentome

     |████████████████████████████████| 1.7MB 7.7MB/s 


In [2]:
!pip install alive-progress

In [3]:
# !wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
# !dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
# !apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
# !apt-get update
# !apt-get install cuda

In [4]:
# !pip install mxnet-cu92

In [32]:
import math
import numpy as np
import pandas as pd
from torchtext.data import Field, BucketIterator, TabularDataset
import random
import json
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from typing import Tuple


import torch.nn.functional as F
from torch import Tensor


from matplotlib import pyplot as plt

import youtokentome as yttm

In [6]:
print(torch.cuda.device_count())

1


In [7]:
print(torch.cuda.get_device_name(0))

Tesla T4


In [8]:
device = torch.device('cuda:0')

In [9]:
# !pip install --upgrade --force-reinstall -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# !unzip '/content/drive/My Drive/unsupervised.csv.zip'

In [12]:
!unzip '/content/drive/My Drive/qa_data.jsonl.zip'

Archive:  /content/drive/My Drive/qa_data.jsonl.zip
  inflating: qa_data.jsonl           
  inflating: __MACOSX/._qa_data.jsonl  


In [13]:
# questions = pd.read_csv('unsupervised.csv')
# questions.question = questions.question.map(lambda x: x.lower())

qa_data = list()

with open('qa_data.jsonl') as file_object:
    for line in file_object:
        qa_data.append(json.loads(line.strip()))
file_object.close()
qa_data = qa_data[:math.ceil(len(qa_data)*0.4)]#!!!!

In [14]:
with open('for_bpe.txt', 'w', encoding='utf-8') as f:
    for que in qa_data:
        f.write(que['question'] + '\n')
f.close()

vocab_size = 30_000
model_path = 'pretrained_bpe_lm.model'

In [15]:
%%time
# обучаем
# раскомментируйте этот код, чтобы обучить bpe
yttm.BPE.train(data='for_bpe.txt', vocab_size=vocab_size, model=model_path)

CPU times: user 9.71 s, sys: 1.11 s, total: 10.8 s
Wall time: 6.7 s


In [16]:
# загружаем токенизатор
tokenizer = yttm.BPE(model=model_path)

In [17]:
qa_data = qa_data[:math.ceil(len(qa_data)*0.6)]

In [18]:
my_pairs = []
max_length = 32
for i in qa_data:
  quest = i['question']
  for resp in i['responses']:
    pair = (quest, resp)
    # pair.append(tok_quest, tokenizer.encode(resp, bos=True, eos=True))
    my_pairs.append(pair)

In [19]:
# tok_pairs = []
# max_length = 32
# for i in qa_data:
  
#   tok_quest = tokenizer.encode(i['question'], eos=True)
#   for resp in i['responses']:
#     pair = (tok_quest, tokenizer.encode(resp, eos=True))
#     # pair.append(tok_quest, tokenizer.encode(resp, bos=True, eos=True))
#     tok_pairs.append(pair)


In [20]:
import pandas as pd 

In [21]:
my_frame = pd.DataFrame(my_pairs)

In [22]:
my_frame.columns = ['q', 'ans']
my_frame = my_frame[:4000]
my_frame

,q,ans
0,долго ли идут деньги с яндексденег на карту visa?,нет. прорыв 35 ;)
1,можно ли зарегистрировать авто в другом регионе,можно на родственника из того региона.. . а п...
2,что делать если у меня очень тонкие ногти а хо...,витамины и умная эмаль (каждый день)
3,что делать если у меня очень тонкие ногти а хо...,ванночки с морской солью. с вечера мажь ногти ...
4,что делать если у меня очень тонкие ногти а хо...,"умная эмаль, витамины, йод, и поменьше крась л..."
...,...,...
3995,моя дочь хочет в художественную гимнастику ей ...,нет! поздно! отдайте в бальные танцы.
3996,какие лучше купить витамины для 17-летней дево...,"витамины фирм ""амвей"", ""арго"""
3997,какие лучше купить витамины для 17-летней дево...,алфавит - ничего лучше раздельного приёма ещё ...
3998,что вы хотели бы забыть ?,забыть что старость приближаетца


In [23]:
my_frame.to_csv('my_frame.csv', index=False)  

In [24]:
max_length = 20
def tokenize(text):
  return tokenizer.encode(text, output_type=yttm.OutputType.SUBWORD)[:max_length]

In [25]:
SRC = Field(tokenize=tokenize, init_token='<sos>', eos_token='<eos>')
TRG = Field(tokenize=tokenize, init_token='<sos>', eos_token='<eos>')

In [26]:
dataset = TabularDataset(path='my_frame.csv', 
                         format='csv', 
                         fields=[ ('ans', TRG), ('q', SRC),],
                         skip_header=True)

In [27]:
train_data, valid_data, test_data = dataset.split(split_ratio=[0.7, 0.1, 0.2], 
                                            random_state=random.getstate())

In [28]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [29]:
print(vars(train_data.examples[0])['ans'])

['▁вопрос?', 'где', '▁родились', '▁братья', '▁самой', 'ло', 'вы']


In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 256

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     sort_key=lambda x: len(x.ans), 
     sort_within_batch=False,
     device = device)

In [ ]:
print (len(SRC.vocab), len(TRG.vocab))
print (SRC.vocab.freqs.most_common(10))
print (TRG.vocab.freqs.most_common(10))

4087 4662
[('▁не', 763), ('▁и', 724), ('▁в', 564), ('.', 515), ('▁а', 390), ('▁на', 365), (',', 269), ('!', 269), ('▁что', 267), ('▁это', 255)]
[('▁в', 647), ('▁как', 499), ('▁на', 495), ('▁что', 401), ('▁и', 401), ('▁не', 367), ('▁а', 362), ('▁с', 324), ('▁вы', 305), ('▁ли', 233)]


In [33]:
class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: Tensor) -> Tuple[Tensor]:

        embedded = self.dropout(self.embedding(src))

        outputs, hidden = self.rnn(embedded)

        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        return outputs, hidden


In [34]:
class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(energy, dim=2)

        return F.softmax(attention, dim=1)

In [35]:
class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:

        a = self.attention(decoder_hidden, encoder_outputs)

        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        return weighted_encoder_rep


    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:

        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        return output, decoder_hidden.squeeze(0)

In [36]:
class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs

In [40]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# HID_DIM = 512
# N_LAYERS = 2
# ENC_DROPOUT = 0.6
# DEC_DROPOUT = 0.6


# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# ENC_HID_DIM = 512
# DEC_HID_DIM = 512
# ATTN_DIM = 64
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [41]:
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights)

optimizer = optim.Adam(model.parameters())


def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,443,000 trainable parameters


In [45]:
PAD_IDX = SRC.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [ ]:
# optimizer = optim.Adam(model.parameters())

# TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
# pad_idx = TRG.vocab.stoi['<pad>']
# print(TRG.pad_token)  
# print(TRG.vocab.stoi[TRG.pad_token]) 

# criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

<pad>
1


In [57]:
def train(model: nn.Module,
          iterator: torch.utils.data.DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for i, batch in enumerate(iterator):

        src = batch.q
        trg = batch.ans

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [58]:
def evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for i, batch in enumerate(iterator):

            src = batch.q
            trg = batch.ans

            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [59]:
def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

In [60]:
import time

In [62]:
for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

Epoch: 01 | Time: 0m 1s
	Train Loss: 7.018 | Train PPL: 1116.369
	 Val. Loss: 6.645 |  Val. PPL: 768.878
Epoch: 02 | Time: 0m 1s
	Train Loss: 7.000 | Train PPL: 1096.677
	 Val. Loss: 6.636 |  Val. PPL: 762.102
Epoch: 03 | Time: 0m 1s
	Train Loss: 6.983 | Train PPL: 1077.970
	 Val. Loss: 6.625 |  Val. PPL: 753.758
Epoch: 04 | Time: 0m 1s
	Train Loss: 6.964 | Train PPL: 1058.242
	 Val. Loss: 6.610 |  Val. PPL: 742.550
Epoch: 05 | Time: 0m 1s
	Train Loss: 6.948 | Train PPL: 1040.847
	 Val. Loss: 6.604 |  Val. PPL: 737.879
Epoch: 06 | Time: 0m 1s
	Train Loss: 6.920 | Train PPL: 1012.082
	 Val. Loss: 6.599 |  Val. PPL: 734.392
Epoch: 07 | Time: 0m 1s
	Train Loss: 6.900 | Train PPL: 992.421
	 Val. Loss: 6.583 |  Val. PPL: 722.742
Epoch: 08 | Time: 0m 1s
	Train Loss: 6.870 | Train PPL: 963.369
	 Val. Loss: 6.582 |  Val. PPL: 721.867
Epoch: 09 | Time: 0m 1s
	Train Loss: 6.839 | Train PPL: 933.243
	 Val. Loss: 6.580 |  Val. PPL: 720.472
Epoch: 10 | Time: 0m 1s
	Train Loss: 6.805 | Train PPL: 90

In [50]:
N_EPOCHS = 1
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    # print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    # print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

NameError: ignored

In [ ]:
def translate_sentence(sentence,src_field,trg_field,model,device, max_len=50):
    model.eval()

    if isinstance(sentence,str):
        tokens = tokenizer.encode(sentence, output_type=yttm.OutputType.SUBWORD)
    else:
        tokens = tokenizer.encode(sentence, output_type=yttm.OutputType.SUBWORD)

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    #
    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)
        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    return trg_tokens[1:-1] # remove <sos> and <eos>

In [ ]:
example_idx = 24

src = vars(train_data.examples[example_idx])['q']
src = " ".join(src)
trg = vars(train_data.examples[example_idx])['ans']

print(f'src = {src}')
print(f'trg = {trg}')

translation = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

src = ▁бро сь ▁его, ▁пока ▁не ▁поздно ! ▁его ▁не ▁передела ешь !!!
trg = ['▁как', '▁нужно', '▁поступить', '▁с', '▁мужчиной,', '▁который', '▁претен', 'дует', '▁на', '▁твоё', '▁личное', '▁простран', 'ство', '▁дик', 'ту', 'я', '▁свои', '▁условия?', ')))']
predicted trg = ['▁как', '▁в', '▁в']


In [ ]:
translation = translate_sentence('что делать', SRC, TRG, model, device)

print(f'predicted trg = {translation}')

translation = translate_sentence('зачем мне это все?', SRC, TRG, model, device)

print(f'predicted trg = {translation}')

translation = translate_sentence('это не нормально, памагити', SRC, TRG, model, device)

print(f'predicted trg = {translation}')

translation = translate_sentence('зачем?', SRC, TRG, model, device)

print(f'predicted trg = {translation}')

translation = translate_sentence('что делать, если холодно', SRC, TRG, model, device)

print(f'predicted trg = {translation}')

predicted trg = ['▁как', '<unk>']
predicted trg = ['▁как', '▁в']
predicted trg = ['▁как', '▁в']
predicted trg = ['<unk>', '<unk>']
predicted trg = ['▁как', '<unk>']
